In [13]:
import pandas as pd
import plotly.express as px

from ddeserts.annotate import add_dvap_columns, add_has_charter_column
from ddeserts.load import load_cvap_data, load_charter_cities

from pandas.api.types import CategoricalDtype

import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
# read in list of charter cities
charter_cities = load_charter_cities()

In [3]:
# load the data
pre_filter = lambda line: 'California' in line
filter = lambda row: (row['state'] == 'California' and 
                      row['geotype'] != 'cdp' and
                      row['lnnumber'] >= 3)

df = load_cvap_data('Place', pre_filter=pre_filter, filter=filter)

In [4]:
# add race category
cats = df[df['name'] == 'Adelanto']['lntitle']
race_cat_type = CategoricalDtype(cats, ordered=True)
df['lntitle'] = df['lntitle'].astype(race_cat_type)

In [9]:
# add DVAP columns
add_dvap_columns(df)
pass

In [11]:
richmond = df.loc[df['name'] == 'Richmond']
richmond

geoname  \
table line                               
Place 39744  Richmond city, California   
      39745  Richmond city, California   
      39746  Richmond city, California   
      39747  Richmond city, California   
      39748  Richmond city, California   
      39749  Richmond city, California   
      39750  Richmond city, California   
      39751  Richmond city, California   
      39752  Richmond city, California   
      39753  Richmond city, California   
      39754  Richmond city, California   

                                                       lntitle  \
table line                                                       
Place 39744             American Indian or Alaska Native Alone   
      39745                                        Asian Alone   
      39746                    Black or African American Alone   
      39747    Native Hawaiian or Other Pacific Islander Alone   
      39748                                        White Alone   
      39749         American Indian or Alaska Native and White   
      39750                                    Asian and White   
      39751                Black or African American and White   
      39752  American Indian or Alaska Native and Black or ...   
      39753            Remainder of Two or More Race Responses   
      39754                                 Hispanic or Latino   

                      geoid  lnnumber  tot_est  tot_moe  adu_est  adu_moe  \
table line                                                                  
Place 39744  16000US0660620         3      325      145      310      144   
      39745  16000US0660620         4    16920     1477    14415     1190   
      39746  16000US0660620         5    21670     1240    18500      978   
      39747  16000US0660620         6      355      263      230      143   
      39748  16000US0660620         7    20025     1020    17815      866   
      39749  16000US0660620         8      360      148      300      120   
      39750  16000US0660620         9      805      235      390      110   
      39751  16000US0660620        10     1300      440      830      257   
      39752  16000US0660620        11      480      217      480      217   
      39753  16000US0660620        12      980      323      705      228   
      39754  16000US0660620        13    46665     1618    31955     1207   

             cit_est  cit_moe  cvap_est  cvap_moe       state      name  \
table line                                                                
Place 39744      325      145       310       144  California  Richmond   
      39745    13375     1182     11030       932  California  Richmond   
      39746    20560     1085     17640       890  California  Richmond   
      39747      325      256       200       122  California  Richmond   
      39748    18705      998     16630       888  California  Richmond   
      39749      360      148       300       120  California  Richmond   
      39750      790      232       375       112  California  Richmond   
      39751     1270      440       800       258  California  Richmond   
      39752      480      217       480       217  California  Richmond   
      39753      940      317       665       218  California  Richmond   
      39754    30630     1394     16880      1120  California  Richmond   

            geotype  dvap_est  dvap_moe  dvap_pct  dvap_pct_moe  
table line                                                       
Place 39744    city         0       204  0.000000      1.228916  
      39745    city      3385      1512  0.234825      0.135459  
      39746    city       860      1323  0.046486      0.078100  
      39747    city        30       188  0.130435      2.375312  
      39748    city      1185      1241  0.066517      0.076618  
      39749    city         0       170  0.000000      0.944444  
      39750    city        15       157  0.038462      0.575824  
      39751    city        30       365  0.036145    

In [26]:
labels = richmond['lntitle'].cat.categories

fig = make_subplots(
    rows=2, cols=1, 
    specs=[[dict(type="pie")]] * 2,
    subplot_titles=["Allowed to Vote", "Disenfranchised"]
)


fig.add_trace(
    go.Pie(
        values=richmond['cvap_est'],
        labels=labels,
        scalegroup='one',
        sort=False,
        hole=0.4,
        direction='clockwise',
        name="Allowed to Vote",
    ), 
    row=1, col=1,
)

fig.add_trace(
    go.Pie(
        values=richmond['dvap_est'],
        labels=labels,
        scalegroup='one',
        sort=False,
        hole=0.4,
        direction='clockwise',
        name="Disenfranchised",
    ), 
    row=2, col=1,
)

fig.update_traces(
    marker=dict(
        line=dict(color='#ffffff', width=2)
    ),
    texttemplate='<b>%{percent:.0%}</b>'
)

fig.update_layout(
    title_font_size=20,
    title_text='Adult Residents by Race/Ethnicity<br><sup>Richmond, CA</sup>',
    title_x=0.5,
    font_family='Raleway',
    legend=dict(traceorder='normal'),
)

fig.update_annotations(font_size=18)

for ann in fig['layout']['annotations']:
    ann['y'] -= 0.05

for pie in fig['data']:
    pie['domain']['y'] = [0.0, 0.9]

fig